# 🎮 Battle City x DreamerV3 (SheepRL)

**Runtime:** T4 GPU recommended

In [ ]:
# 1. Clone repo
!rm -rf /content/battle_city
!git clone https://github.com/Nikolay1221/x.git /content/battle_city
%cd /content/battle_city
!ls

In [ ]:
# 2. System deps
!apt-get update -qq
!apt-get install -y -qq freeglut3-dev xvfb

In [ ]:
# 3. Python packages
!pip install -q gymnasium opencv-python matplotlib nes-py
!pip install -q lightning sheeprl
print('✅ Packages installed')

In [ ]:
# 4. Start virtual display
!Xvfb :1 -screen 0 1024x768x24 &
import os
os.environ['DISPLAY'] = ':1'
print('✅ Virtual display ready')

In [ ]:
# 5. Test environment
%cd /content/battle_city
!python train_dreamer.py

In [ ]:
# 6. Train with SheepRL
%cd /content/battle_city

# Register env first
import sys
sys.path.insert(0, '/content/battle_city')

import gymnasium as gym
from gymnasium.envs.registration import register
import numpy as np
from battle_city_env import BattleCityEnv
from dreamer_wrapper import BattleCityDreamerWrapper

class BattleCityGymEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self._env = BattleCityEnv(rom_path='/content/battle_city/BattleCity.nes')
        self._wrapper = BattleCityDreamerWrapper(self._env, size=(64, 64))
        self.observation_space = gym.spaces.Box(0, 255, (64, 64, 1), dtype=np.uint8)
        self.action_space = gym.spaces.Discrete(10)

    def reset(self, **kw):
        obs, info = self._wrapper.reset(**kw)
        return obs['image'], info

    def step(self, action):
        obs, r, t, tr, info = self._wrapper.step(action)
        return obs['image'], float(r), t, tr, info

    def close(self):
        self._env.close()

register(id='BattleCity-v0', entry_point=lambda: BattleCityGymEnv())

# Run sheeprl
!python -m sheeprl exp=dreamer_v3 env.id=BattleCity-v0 fabric.devices=1 algo.total_steps=100000